<h1> CSCI 567 - Machine Learning - Spring 2021 </h1>
<h2> Project: Chekcpoint 1 - Data Preprocessing (03/10/21) </h2>
<h3> Team <u>StochasticResults</u>: </h3>
 - Abel Salinas [8793999216] [abelsali@usc.edu] <br>
 - Angel Nieto [2211798052] [nietogar@usc.edu] <br>
 - Misael Morales [5832732058] [misaelmo@usc.edu]

***

The "Housing Price" dataset consists of 79 predictors for the house prices in Ames, Iowa. The training and testing set are already pre-split for us from the Kaggle version (https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview), and we will focus on different regression techniques to predict the housing SalePrice based on the given features.

This notebook focuses on the data pre-processing, wrangling, visualization, and statistical analysis. It is a crucial step in any machine-learning/data-analytics application to ensure proper data formatting in order to optimize the techniques implemented. 

# Table of Contents:
1. Load required packages <br>
2. Data wrangling <br>
3. Data visualization <br>
4. Exploratory data analysis

***

# 1. Load required libraries

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
# Basic data management packages
import os
import numpy as np
import pandas as pd

# Visualization packages
import matplotlib.pyplot as plt
import seaborn as sns
import pandas.plotting as pd_plot
%matplotlib inline

In [3]:
# Exploratory Data Analysis packages
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA  
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import LocalOutlierFactor

In [4]:
# Regression and Modeling packages
import tensorflow as tf
import keras
from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score

# Verify GPU compatibility
print("Tensorflow Version:", tf.__version__)
print("Tensorflow built with CUDA?", tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices('CPU'))
print(tf.config.list_physical_devices('GPU'))
print("Num GPU Available:", len(tf.config.list_physical_devices('GPU')))

Tensorflow Version: 2.4.1
Tensorflow built with CUDA? False
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
[]
Num GPU Available: 0


In [5]:
# Preprocessing choice
# preprocessing_method = 'HousePreprocessor'
preprocessing_method = 'Mappings'

***

In [6]:
# Read CSV files for Train/Test datasets
train_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_df  = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [7]:
print('Train Shape: {} | types: {} \nTest Shape:  {} | types: {}'.format(train_df.shape, pd.unique(train_df.dtypes), 
                                                                       test_df.shape, pd.unique(test_df.dtypes)))
print('Set difference train-vs-test: {}'.format(set(train_df.columns).difference(set(test_df.columns))))

Train Shape: (1460, 81) | types: [dtype('int64') dtype('O') dtype('float64')] 
Test Shape:  (1459, 80) | types: [dtype('int64') dtype('O') dtype('float64')]
Set difference train-vs-test: {'SalePrice'}


In [8]:
x_train = train_df.iloc[:,1:-1]  #79 train features
y_train = train_df.iloc[:,-1]    #SalePrice training target
x_test  = test_df.iloc[:,1:]     #79 test features
print('x_train {} | y_train {} \nx_test  {}'.format(x_train.shape, y_train.shape, x_test.shape))

x_train (1460, 79) | y_train (1460,) 
x_test  (1459, 79)


In [9]:
#preview the training data set
x_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [10]:
#preview the testing data set
x_test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [11]:
numerical_feats = x_train.dtypes[x_train.dtypes != "object"].index
print("Number of Numerical features: ", len(numerical_feats))

categorical_feats = x_train.dtypes[x_train.dtypes == "object"].index
print("Number of Categorical features: ", len(categorical_feats))

Number of Numerical features:  36
Number of Categorical features:  43


In [12]:
total = x_train.isnull().sum().sort_values(ascending=False)
percent = (x_train.isnull().sum()/x_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(10)

,Total,Percent
PoolQC,1453,0.995205
MiscFeature,1406,0.963014
Alley,1369,0.937671
Fence,1179,0.807534
FireplaceQu,690,0.472603
LotFrontage,259,0.177397
GarageType,81,0.055479
GarageYrBlt,81,0.055479
GarageQual,81,0.055479
GarageCond,81,0.055479


In [13]:
# Non-numeric variables that require attention
non_num_vars = x_train.dtypes[x_train.dtypes=='object'].index
print(non_num_vars)

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')


In [14]:
# Replace object/string values with categorial values from the "data_description" file from Kaggle
mappings = dict(
MSZoning_mapping      = {'nan':0, 'A':1, 'C (all)':2, 'FV':3, 'I':4, 'RH':5, 'RL':6, 'RP':7, 'RM':8},
Street_mapping        = {'nan':0, 'Grvl':1, 'Pave':2},
Alley_mapping         = {'nan':0, 'Grvl':1, 'Pave':2, 'NaN':0},
LotShape_mapping      = {'nan':0, 'Reg':1, 'IR1':2, 'IR2':3, 'IR3':4 },
LandContour_mapping   = {'nan':0, 'Lvl':1, 'Bnk':2, 'HLS':3, 'Low':4},
Utilities_mapping     = {'nan':0, 'AllPub':1, 'NoSewr':2, 'NoSeWa':3, 'ELO':4},
LotConfig_mapping     = {'nan':0, 'Inside':1, 'Corner':2, 'CulDSac':3, 'FR2':4, 'FR3':5},
LandSlope_mapping     = {'nan':0, 'Gtl':1, 'Mod':2, 'Sev':3},
Neighborhood_mapping  = {'nan':0, 'Blmngtn':1, 'Blueste':2, 'BrDale':3, 'BrkSide':4, 'ClearCr':5, 'CollgCr':6, 'Crawfor':7,
                         'Edwards':8, 'Gilbert':9, 'IDOTRR':10, 'MeadowV':11, 'Mitchel':12, 'NAmes':13, 'NoRidge':14, 
                         'NPkVill':15, 'NridgHt':16, 'NWAmes':17, 'OldTown':18, 'SWISU':19, 'Sawyer':20, 'SawyerW':21, 
                         'Somerst':22, 'StoneBr':23, 'Timber':24, 'Veenker':25},
Condition1_mapping    = {'nan':0, 'Artery':1, 'Feedr':2, 'Norm':3, 'RRNn':4, 'RRAn':5, 'PosN':6, 'PosA':7, 'RRNe':8, 'RRAe':9},
Condition2_mapping    = {'nan':0, 'Artery':1, 'Feedr':2, 'Norm':3, 'RRNn':4, 'RRAn':5, 'PosN':6, 'PosA':7, 'RRNe':8, 'RRAe':9},
BldgType_mapping      = {'nan':0, '1Fam':1, '2fmCon':2, 'Duplex':3, 'Twnhs':4, 'TwnhsE':4, 'TwnhsI':5},
HouseStyle_mapping    = {'nan':0, '1Story':1, '1.5Fin':2, '1.5Unf':3, '2Story':4, '2.5Fin':5, '2.5Unf':6, 'SFoyer':7, 'SLvl':8},
RoofStyle_mapping     = {'nan':0, 'Flat':1, 'Gable':2, 'Gambrel':3, 'Hip':4, 'Mansard':5, 'Shed':6},
RoofMatl_mapping      = {'nan':0, 'ClyTile':1, 'CompShg':2, 'Membran':3, 'Metal':4, 
                         'Roll':5, 'Tar&Grv':6, 'WdShake':7, 'WdShngl':8},
Exterior1st_mapping   = {'nan':0, 'AsbShng':1, 'AsphShn':2, 'BrkComm':3, 'BrkFace':4, 'CBlock':5, 'CemntBd':6, 
                         'HdBoard':7, 'ImStucc':8, 'MetalSd':9, 'Other':10, 'Plywood':11, 'PreCast':12, 'Stone':13, 
                         'Stucco':14, 'VinylSd':15,'Wd Sdng':16, 'WdShing':17},
Exterior2nd_mapping   = {'nan':0, 'AsbShng':1, 'AsphShn':2, 'Brk Cmn':3, 'BrkFace':4, 'CBlock':5, 'CmentBd':6, 
                         'HdBoard':7, 'ImStucc':8, 'MetalSd':9, 'Other':10, 'Plywood':11, 'PreCast':12, 'Stone':13, 
                         'Stucco':14, 'VinylSd':15, 'Wd Shng':16, 'Wd Sdng':16, 'WdShing':17},
MasVnrType_mapping    = {'nan':0, 'BrkCmn':1, 'BrkFace':2, 'CBlock':3, 'None':4, 'Stone':5},
ExterQual_mapping     = {'nan':0, 'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po':5},
ExterCond_mapping     = {'nan':0, 'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po':5},
Foundation_mapping    = {'nan':0, 'BrkTil':1, 'CBlock':2, 'PConc':3, 'Slab':4, 'Stone':5, 'Wood':6},
BsmtQual_mapping      = {'nan':0, 'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po':5, 'NA':0},
BsmtCond_mapping      = {'nan':0, 'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po':5, 'NA':0},
BsmtExposure_mapping  = {'nan':0, 'Gd':1, 'Av':2, 'Mn':3, 'No':4, 'NA':5},
BsmtFinType1_mapping  = {'nan':0, 'GLQ':1, 'ALQ':2, 'BLQ':3, 'Rec':4, 'LwQ':5, 'Unf':6, 'NA':0},
BsmtFinType2_mapping  = {'nan':0, 'GLQ':1, 'ALQ':2, 'BLQ':3, 'Rec':4, 'LwQ':5, 'Unf':6, 'NA':0},
Heating_mapping       = {'nan':0, 'Floor':1, 'GasA':2, 'GasW':3, 'Grav':4, 'OthW':5, 'Wall':6},
HeatingQC_mapping     = {'nan':0, 'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po':5},
CentralAir_mapping    = {'nan':0, 'N':1, 'Y':2},
Electrical_mapping    = {'nan':0, 'SBrkr':1, 'FuseA':2, 'FuseF':3, 'FuseP':4, 'Mix':5},
KitchenQual_mapping   = {'nan':0, 'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po':5},
Functional_mapping    = {'nan':0, 'Typ':1, 'Min1':2, 'Min2':3, 'Mod':4, 'Maj1':5, 'Maj2':6, 'Sev':7, 'Sal':8},
FireplaceQu_mapping   = {'nan':0, 'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po':5, 'NA':0},
GarageType_mapping    = {'nan':0, '2Types':1, 'Attchd':2, 'Basment':3, 'BuiltIn':4, 'CarPort':5, 'Detchd':6, 'NA':0},
GarageFinish_mapping  = {'nan':0, 'Fin':1, 'RFn':2, 'Unf':3, 'NA':0},
GarageQual_mapping    = {'nan':0, 'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po':5, 'NA':0},
GarageCond_mapping    = {'nan':0, 'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po':5, 'NA':0},
PavedDrive_mapping    = {'nan':0, 'Y':1, 'P':2, 'N':3},
PoolQC_mapping        = {'nan':0, 'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'NA':0},
Fence_mapping         = {'nan':0, 'GdPrv':1, 'MnPrv':2, 'GdWo':3, 'MnWw':4, 'NA':0},
MiscFeature_mapping   = {'nan':0, 'Elev':1, 'Gar2':2, 'Othr':3, 'Shed':4, 'TenC':5, 'NA':0},
SaleType_mapping      = {'nan':0, 'WD':1, 'CWD':2, 'VWD':3, 'New':4, 'COD':5, 'Con':6, 'ConLw':7, 'ConLI':8,
                         'ConLD':9, 'Oth':10},
SaleCondition_mapping = {'nan':0, 'Normal':1, 'Abnorml':2, 'AdjLand':3, 'Alloca':4, 'Family':5, 'Partial':6})

In [15]:
x_train = x_train.replace({list(non_num_vars)[k] : list(mappings.values())[k] 
                           for k in np.arange(len(list(non_num_vars)))}).fillna(0)
x_test  = x_test.replace({list(non_num_vars)[k] : list(mappings.values())[k] 
                           for k in np.arange(len(list(non_num_vars)))}).fillna(0)

print('x_train {} | y_train {} \nx_test  {}'.format(x_train.shape, y_train.shape, x_test.shape))

x_train (1460, 79) | y_train (1460,) 
x_test  (1459, 79)


In [16]:
numerical_feats = x_train.dtypes[x_train.dtypes != "object"].index
print("Number of Numerical features: ", len(numerical_feats))

categorical_feats = x_train.dtypes[x_train.dtypes == "object"].index
print("Number of Categorical features: ", len(categorical_feats))

Number of Numerical features:  79
Number of Categorical features:  0


***

In [17]:
corr_mat = np.corrcoef(x_train.join(y_train),rowvar = False)
corr_vec = corr_mat[:-1,-1]   #correlation of features to target only
print(corr_mat.round(2))            

[[ 1.    0.08 -0.22 ... -0.   -0.04 -0.08]
 [ 0.08  1.   -0.07 ... -0.13 -0.16 -0.21]
 [-0.22 -0.07  1.   ...  0.11  0.18  0.21]
 ...
 [-0.   -0.13  0.11 ...  1.    0.54  0.15]
 [-0.04 -0.16  0.18 ...  0.54  1.    0.29]
 [-0.08 -0.21  0.21 ...  0.15  0.29  1.  ]]


In [18]:
#View features that have a +/- correlation greater than the user-specified corrrelation bound (corr_bound)
corr_bound = 0.6
corr_dat = (x_train.iloc[:, corr_vec<-corr_bound]).join(x_train.iloc[:, corr_vec>corr_bound])
corr_dat

,ExterQual,KitchenQual,OverallQual,TotalBsmtSF,1stFlrSF,GrLivArea,GarageCars,GarageArea
0,2,2,7,856,856,1710,2,548
1,3,3,6,1262,1262,1262,2,460
2,2,2,7,920,920,1786,2,608
3,3,2,7,756,961,1717,3,642
4,2,2,8,1145,1145,2198,3,836
...,...,...,...,...,...,...,...,...
1455,3,3,6,953,953,1647,2,460
1456,3,3,6,1542,2073,2073,2,500
1457,1,2,7,1152,1188,2340,1,252
1458,3,2,5,1078,1078,1078,1,240


In [19]:
#full training set basic statistics
x_train.describe()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,6.126712,57.623288,10516.828082,1.995890,0.090411,1.408219,1.185616,1.001370,1.416438,...,15.060959,2.758904,0.010959,0.396575,0.144521,43.489041,6.321918,2007.815753,1.490411,1.582192
std,42.300571,1.050330,34.664304,9981.264932,0.063996,0.372151,0.582296,0.606509,0.052342,0.773448,...,55.757415,40.177307,0.177224,0.875914,0.742569,496.123024,2.703626,1.328095,1.368616,1.475209
min,20.000000,2.000000,0.000000,1300.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,1.000000,1.000000
25%,20.000000,6.000000,42.000000,7553.500000,2.000000,0.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,1.000000,1.000000
50%,50.000000,6.000000,63.000000,9478.500000,2.000000,0.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,1.000000,1.000000
75%,70.000000,6.000000,79.000000,11601.500000,2.000000,0.000000,2.000000,1.000000,1.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,1.000000,1.000000
max,190.000000,8.000000,313.000000,215245.000000,2.000000,2.000000,4.000000,4.000000,3.000000,5.000000,...,480.000000,738.000000,4.000000,4.000000,5.000000,15500.000000,12.000000,2010.000000,10.000000,6.000000


In [20]:
#main features (based on correlation coefficient) basic statistics
corr_dat.describe()

,ExterQual,KitchenQual,OverallQual,TotalBsmtSF,1stFlrSF,GrLivArea,GarageCars,GarageArea
count,1460.00000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,2.60411,2.488356,6.099315,1057.429452,1162.626712,1515.463699,1.767123,472.980137
std,0.57428,0.663760,1.382997,438.705324,386.587738,525.480383,0.747315,213.804841
min,1.00000,1.000000,1.000000,0.000000,334.000000,334.000000,0.000000,0.000000
25%,2.00000,2.000000,5.000000,795.750000,882.000000,1129.500000,1.000000,334.500000
50%,3.00000,3.000000,6.000000,991.500000,1087.000000,1464.000000,2.000000,480.000000
75%,3.00000,3.000000,7.000000,1298.250000,1391.250000,1776.750000,2.000000,576.000000
max,4.00000,4.000000,10.000000,6110.000000,4692.000000,5642.000000,4.000000,1418.000000


In [21]:
corr_dat.join(y_train).corr()['SalePrice']

ExterQual     -0.682639
KitchenQual   -0.659600
OverallQual    0.790982
TotalBsmtSF    0.613581
1stFlrSF       0.605852
GrLivArea      0.708624
GarageCars     0.640409
GarageArea     0.623431
SalePrice      1.000000
Name: SalePrice, dtype: float64

In [22]:
target = np.log10(y_train)
print('       SalePrice: Mean {:.3e} | Std. Dev {:.3e}'.format(y_train.mean(), y_train.std()))
print('Log10(SalePrice): Mean {:.3f}     | Std. Dev {:.3f}'.format(target.mean(), target.std()))

       SalePrice: Mean 1.809e+05 | Std. Dev 7.944e+04
Log10(SalePrice): Mean 5.222     | Std. Dev 0.173


In [23]:
target_name = 'SalePrice'

corr_abs = train_df.corr().abs()
ser_corr = corr_abs.nlargest(len(numerical_feats), target_name)[target_name]

cols_abv_corr_limit = list(ser_corr[ser_corr.values >= corr_bound].index)
cols_bel_corr_limit = list(ser_corr[ser_corr.values < corr_bound].index)

nr_feats = len(cols_abv_corr_limit)

In [24]:
scaler = StandardScaler()
x_train_s = scaler.fit_transform(corr_dat)

In [25]:
n_components = 4
pca = PCA(n_components=n_components)
pca.fit(x_train_s)

#variance explained by each PC
print('PC Variance Explained: {}'.format(pca.explained_variance_ratio_))
print('Cumulative Variance Explained: {}'.format(np.sum(pca.explained_variance_ratio_).round(4)))

PC Variance Explained: [0.59394484 0.12746352 0.10382664 0.07290565]
Cumulative Variance Explained: 0.8981


In [26]:
x_trans = pca.transform(x_train_s)  

In [27]:
n_clusters = 3
kmeans = KMeans(n_clusters = n_clusters).fit(X=x_train_s, y=target).labels_
hclust = AgglomerativeClustering(n_clusters = n_clusters).fit(X=x_train_s, y=target).labels_
gmm    = GaussianMixture(n_components=n_clusters).fit_predict(X=x_train_s, y=target)

In [28]:
df_all = pd.concat([x_train, x_test])

len_train = x_train.shape[0]

In [29]:
# StandardScaler from Scikit-Learn
from sklearn.preprocessing import StandardScaler

# Initialize instance of StandardScaler
scaler = StandardScaler()

# Fit and transform item_data
data_scaled = scaler.fit_transform(df_all)

# Display first 5 rows of item_data_scaled
data_scaled[:5]

array([[ 0.06733141, -0.10861873,  0.21607482, -0.21787869,  0.06424926,
        -0.25355997, -0.70189976, -0.30847126,  0.        , -0.52215438,
        -0.21626042, -1.24756925, -0.05704881, -0.02099186, -0.43148064,
         0.74057214,  0.6461834 , -0.5072842 ,  1.0462576 ,  0.89683293,
        -0.48292558, -0.11692303,  0.79453692,  0.79756559, -1.31848566,
         0.52903417, -1.03980539,  0.23004678,  0.834618  , -0.44745004,
         0.15158359,  0.65275866, -1.07980856,  0.58114542,  0.35402201,
        -0.29302528, -0.93416481, -0.44307791, -0.1032061 , -0.88561933,
         0.26828983, -0.27700532, -0.77386103,  1.20737855, -0.10119702,
         0.41354722,  1.08733408, -0.24971861,  0.78136581,  1.23259887,
         0.16992709, -0.20769847, -0.73567911,  0.98684937, -0.23598698,
        -0.92431121, -0.94402138, -0.58514876,  0.29500243, -0.07043436,
         0.30703976,  0.34936362,  0.17460855,  0.18712257, -0.31502962,
        -0.74076041,  0.20000601, -0.35960075, -0.1

In [30]:
pca = PCA(n_components=n_components)
pca.fit(data_scaled)

PC_items = pca.transform(data_scaled)

In [31]:
# Put PC_items into a dataframe
items_pca = pd.DataFrame(PC_items)

# Name the columns
items_pca.columns = ['PC{}'.format(i + 1) for i in range(PC_items.shape[1])]

# Update its index
items_pca.index = df_all.index

# Display first 5 rows
items_pca.head()

,PC1,PC2,PC3,PC4
0,2.111772,0.621553,-0.812348,-1.488850
1,0.410106,-1.111971,1.858412,0.965361
2,2.806540,0.315364,-0.529899,-0.746065
3,-0.776157,-1.770408,-1.645372,0.967551
4,5.131369,0.237021,-1.246777,0.985085


In [32]:
len_train = x_train.shape[0]

pca_xtrain = items_pca[:len_train]
pca_xtest = items_pca[len_train:]

In [33]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(x_train, y_train)
pred = lr.predict(x_test)
pred_train = lr.predict(x_train)
r2 = r2_score(y_train,pred_train)
rmse = np.sqrt(mean_squared_error(y_train,pred_train))

In [34]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

In [35]:
print( 'R^2:', r2_score(y_train, pred_train ))
print( 'MAE:', mean_absolute_error(y_train, pred_train))

R^2: 0.8496524162299119
MAE: 19206.088668798126


In [36]:
from sklearn.ensemble import RandomForestRegressor

In [37]:
# rf = RandomForestRegressor() #default features
# rf_fit = rf.fit(x_train, y_train)
# pred = rf_fit.predict(x_test)
# pred_train = rf.predict(x_train)
# r2 = r2_score(y_train,pred_train)
# rmse = np.sqrt(mean_squared_error(y_train,pred_train))
# print( 'R^2:', r2_score(y_train, pred_train ))
# print( 'MAE:', mean_absolute_error(y_train, pred_train))

In [38]:
import xgboost
reg = xgboost.XGBRegressor().fit(x_train, y_train)
pred = reg.predict(x_test)
pred_train = reg.predict(x_train)
r2 = r2_score(y_train,pred_train)
rmse = np.sqrt(mean_squared_error(y_train,pred_train))
print( 'R^2:', r2_score(y_train, pred_train ))
print( 'MAE:', mean_absolute_error(y_train, pred_train))

R^2: 0.9996259533146924
MAE: 1072.3712837114726


In [39]:
id_test = test_df['Id']

pred_pd = pd.DataFrame()
pred_pd['Id'] = id_test
pred_pd['SalePrice'] = pred

pred_pd.head
pred_pd.to_csv('submission_StochasticResults_RF.csv',index=False)

***

***

# End of Notebook